<a href="https://colab.research.google.com/github/juanfranbrv/curso-langchain/blob/main/Splitting%20Text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Text Splitting**
---
El **Text Splitting** (o división de texto) es una etapa crítica en los sistemas de **Retrieval-Augmented Generation (RAG)**. Consiste en fragmentar documentos extensos en segmentos más pequeños y manejables (_chunks_) antes de indexarlos para su recuperación posterior. Su implementación incide directamente en la calidad de la recuperación de información y, por ende, en las respuestas generadas por el modelo.

* * *

### **¿Por qué es importante?**

1.  **Equilibrio entre contexto y precisión**:
    
    -   **Chunks demasiado grandes**: Pueden contener información redundante o irrelevante, lo que "diluye" el contexto clave y reduce la precisión del retrieval.
        
    -   **Chunks demasiado pequeños**: Pierden contexto necesario para entender el significado completo (p. ej., una frase sin su párrafo asociado).
        
    -   Un buen _splitting_ mantiene la coherencia semántica en cada segmento, facilitando que el sistema recupere los fragmentos más relevantes para una consulta.
        
2.  **Compatibilidad con modelos de embedding**:
    
    -   Los modelos de embedding (como SBERT o OpenAI) tienen límites óptimos de longitud de texto. Por ejemplo, un chunk de 512 tokens funciona bien con muchos codificadores, pero un texto más largo podría truncarse o perder información crítica.
        
3.  **Impacto en la generación de respuestas**:
    
    -   Los chunks recuperados alimentan al modelo generador (como GPT-4). Si están mal estructurados, el modelo recibirá información fragmentada o fuera de contexto, lo que generará respuestas inconsistentes o inexactas.
        
4.  **Eficiencia computacional**:
    
    -   Chunks bien dimensionados reducen costos de procesamiento y latencia, ya que evitan sobrecargar el sistema con datos innecesarios.


### El Text Splitting no es un paso mecánico, sino una decisión estratégica que determina cómo el sistema "ve" la información y afectara enormemente a la calidad del sistema.

# **Nivel 1: División por Caracteres**  
---
La **división por caracteres** es la forma más básica de fragmentar texto. Consiste simplemente en dividir el texto en _chunks_ (segmentos) de un tamaño fijo de **N caracteres**, ignorando por completo el contenido o estructura del texto.

Este método **no se recomienda para aplicaciones reales**, pero es un punto de partida útil para comprender los fundamentos del _text splitting_.

**Ventajas**:

-   Fácil y sencillo de implementar.
    

**Desventajas**:

-   Muy rígido: **no considera la estructura del texto** (p. ej., separación de párrafos, puntuación o temas).
    
-   Puede fragmentar ideas o contextos clave a la mitad.
    

**Conceptos clave**:

-   **Tamaño del chunk (_Chunk Size_)**: Número de caracteres por segmento (ej: 50, 100, 1000).
    
-   **Solapamiento de chunks (_Chunk Overlap_)**: Cantidad de caracteres que se superponen entre chunks consecutivos. Esto ayuda a evitar que una misma información contextual quede dividida en múltiples chunks, aunque genera duplicación de datos.



In [1]:
text = "El text splitting divide textos en segmentos pequeños para facilitar su procesamiento. Es clave mantener el equilibrio entre tamaño y contexto. Un buen splitting mejora la precisión en sistemas como RAG."

In [2]:
# Creamos una lista para almacenar los chunks
chunks = []

chunk_size = 35 # Caracteres

# desde 0 hasta la longitud del texto con salto de chunk_size
for i in range(0, len(text), chunk_size):
    chunk = text[i:i + chunk_size]
    chunks.append(chunk)
chunks

['El text splitting divide textos en ',
 'segmentos pequeños para facilitar s',
 'u procesamiento. Es clave mantener ',
 'el equilibrio entre tamaño y contex',
 'to. Un buen splitting mejora la pre',
 'cisión en sistemas como RAG.']

Langchain tiene una clae que permite realizar esta tarea facilmente

In [7]:
from langchain.text_splitter import CharacterTextSplitter

# Instanciamos un objeto CharacterTextSplitter y le pasamos los pasarmetros de corte
text_splitter = CharacterTextSplitter(chunk_size = 35, chunk_overlap=0, separator='', )

# Then we can actually split our text via create_documents.
# Note: create_documents expects a list of texts, so if you just have a string (like we do) you'll need to wrap it in []
text_splitter.create_documents([text])


[Document(metadata={}, page_content='El text splitting divide textos en'),
 Document(metadata={}, page_content='segmentos pequeños para facilitar s'),
 Document(metadata={}, page_content='u procesamiento. Es clave mantener'),
 Document(metadata={}, page_content='el equilibrio entre tamaño y contex'),
 Document(metadata={}, page_content='to. Un buen splitting mejora la pre'),
 Document(metadata={}, page_content='cisión en sistemas como RAG.')]

Nos devolvera una lista de _documentos_
Observa que, esta vez, tenemos los mismos chunks, pero ahora están en documentos. Estos funcionarán bien con el resto del ecosistema de LangChain. También nota que el espacio en blanco al final del tercer chunk ha desaparecido. Esto se debe a que LangChain lo elimina automáticamente. Si deseas conservar los espacios en blanco, puedes evitarlo configurando strip_whitespace=False.

**Chunk Overlap & Separadores**

El chunk overlap (solapamiento de chunks) fusiona los segmentos de texto de manera que el final del Chunk #1 coincida con el inicio del Chunk #2, y así sucesivamente. Esto ayuda a mantener el contexto entre chunks adyacentes, evitando que se pierda información importante en las divisiones.

En este caso, configuraré un solapamiento de 4 caracteres, lo que significa que los últimos 7 caracteres de un chunk serán los primeros 10 del siguiente.

In [12]:
text_splitter = CharacterTextSplitter(chunk_size = 35, chunk_overlap=10, separator='')

text_splitter.create_documents([text])

[Document(metadata={}, page_content='El text splitting divide textos en'),
 Document(metadata={}, page_content='vide textos en segmentos pequeños p'),
 Document(metadata={}, page_content='ntos pequeños para facilitar su pro'),
 Document(metadata={}, page_content='acilitar su procesamiento. Es clave'),
 Document(metadata={}, page_content='iento. Es clave mantener el equilib'),
 Document(metadata={}, page_content='ener el equilibrio entre tamaño y c'),
 Document(metadata={}, page_content='ntre tamaño y contexto. Un buen spl'),
 Document(metadata={}, page_content='to. Un buen splitting mejora la pre'),
 Document(metadata={}, page_content='g mejora la precisión en sistemas c'),
 Document(metadata={}, page_content='n en sistemas como RAG.')]

Una interesante herramienta para visualizar como funciona este sistema es ChunkViz.com de